In [1]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from pytexit import py2tex
from IPython.display import Markdown
from gekko import GEKKO
import pandas as pd

%matplotlib widget
#%matplotlib notebook

In [8]:
# https://apmonitor.com/pdc/index.php/Main/TCLabSim

TCLab = GEKKO()    # create GEKKO model for dx/dt

t_sim = 300  # Number of 1-second time points (5 min)

TCLab.time = np.linspace(0,t_sim,t_sim+1) # time points

# dt is in the units of ________.

T_Ambient = 22.5 # degC
Tau_p     = 120.0 # seconds

# Starting temperature for the heatsink.
# Might be equal to 'T_Ambient' or something else.
#HS_T_Start = T_Ambient
HS_T_Start = 36.0

K_p       = TCLab.Param()  # ~= 0.8 oC / %
Heater_Q  = TCLab.Param()  # "Percentage of maximum value" of the heater -- set the heater percentage down below 

HS_Temp     = TCLab.Var(HS_T_Start) # GEKKO variable for the temperature of the heatsink

TCLab.Equation(HS_Temp.dt()  == ( (T_Ambient - HS_Temp) + (K_p * Heater_Q) ) / Tau_p )

# solve ODEs and plot
# https://gekko.readthedocs.io/en/latest/imode.html
TCLab.options.IMODE = 4 # dynamic simulation

# Type: Integer, Input/Output
# Default Value: 0
# Description: Cold start model: 0=warm start, 1=cold start, 2=decompose problem
#TCLab.options.COLDSTART = 2

#TCLab.options.SOLVER = 1
#TCLab.options.MAX_ITER = 1000

#---------------------
TCLab.options.NODES = 6
#---------------------

TCLab.options.TIME_SHIFT=0



fig = plt.figure()
plt_ax = plt.axes(label='TCLab')
plt_ax.set_ylim(20,60)

HeaterValue = 50
Heater_Q.value = HeaterValue # Turn the heater on to the specified percentage value
#print(Heater_Q.value)

#k_vals = [10, 20, 30, 50]
K_p_vals = [0.55, 0.65]

for K_p_val in K_p_vals:
    K_p.value = K_p_val
    TCLab.solve(disp=False)
    plt_ax.plot(TCLab.time,HS_Temp, marker='',linewidth=1,label='K_p = {0}'.format(K_p.value[0]))

######################################################################################
# Create the 'dummy' plot with NaN's that will be replaced with measured data below.
T1 = np.empty(t_sim)
T1[:] = np.nan
T1_time = np.empty(t_sim)
T1_time[:] = np.nan
MeasuredData, = plt_ax.plot(T1_time,T1,'r.', label='Measured')
######################################################################################

plt.grid(True)
plt.title('TC Lab Thermal Simulation')
plt.xlabel('Time')
plt.ylabel('Temperature (degC)')
plt.legend()
plt.show()

# Now solve as a steady-state problem in case the simulation time isn't long enough.
# Sometimes the parameter values doen't seem to 'stick'...
#TCLab.options.IMODE = 3
#TCLab.solve(disp=False)
#print('Final Temperature = {0} for a Heater Value of {1}% and a K_p of {2}'.format(HS_Temp.value[0], HeaterValue, K_p.value[0]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
# Actual measured data
import tclab
import time


if (1):
    lab = tclab.TCLab()
    #T1 = [lab.T1] # Old-style list based plotting...
    #lab.Q1(50)
    print('Turning Heater on to {0}%'.format(HeaterValue))
    lab.Q1(HeaterValue)

    T1[:] = np.nan       # Reset the measured temperatures to NaN's
    T1_time[:] = np.nan

    for i in range(t_sim):
        time.sleep(1)
        #print(lab.T1)
        print('T1 = {0}, T2 = {1}'.format(lab.T1, lab.T2))
        T1[i] = lab.T1
        T1_time[i] = i
        MeasuredData.set_ydata(np.array(T1))
        MeasuredData.set_xdata(np.array(T1_time))
        plt_ax.set_title("{0} of {1} Points Measured".format(i+1,t_sim))
        fig.canvas.draw()
        fig.canvas.flush_events()

    lab.close()

TCLab version 0.4.9
Arduino Leonardo connected on port COM3 at 115200 baud.
TCLab Firmware 1.4.3 Arduino Leonardo/Micro.
Turning Heater on to 50%
T1 = 34.12, T2 = 31.54
T1 = 33.8, T2 = 31.54
T1 = 33.8, T2 = 31.22
T1 = 31.86, T2 = 31.54
T1 = 33.8, T2 = 31.54
T1 = 33.8, T2 = 31.22
T1 = 33.8, T2 = 31.22
T1 = 33.48, T2 = 31.22
T1 = 33.8, T2 = 31.22
T1 = 33.48, T2 = 31.22
T1 = 33.48, T2 = 31.22
T1 = 33.8, T2 = 31.22
T1 = 33.8, T2 = 31.22
T1 = 33.8, T2 = 31.22
T1 = 33.8, T2 = 31.22
T1 = 33.8, T2 = 31.22
T1 = 33.8, T2 = 31.22
T1 = 34.12, T2 = 30.9
T1 = 34.12, T2 = 30.9
T1 = 34.12, T2 = 31.22
T1 = 32.19, T2 = 30.9
T1 = 34.12, T2 = 30.9
T1 = 34.12, T2 = 30.9
T1 = 34.12, T2 = 30.9
T1 = 34.44, T2 = 30.9
T1 = 34.44, T2 = 30.9
T1 = 34.76, T2 = 30.9
T1 = 34.76, T2 = 30.9
T1 = 34.76, T2 = 30.9
T1 = 34.76, T2 = 30.9
T1 = 35.09, T2 = 30.9
T1 = 35.09, T2 = 30.9
T1 = 35.09, T2 = 30.9
T1 = 35.41, T2 = 30.9
T1 = 35.41, T2 = 30.9
T1 = 35.41, T2 = 30.57
T1 = 35.41, T2 = 30.57
T1 = 35.41, T2 = 30.57
T1 = 35.7

In [7]:
if (1): # Grab the current temperature from the TCLab if desired. (I.e. = 1)
    lab = tclab.TCLab()
    print('T1 = {0}, T2 = {1}'.format(lab.T1, lab.T2))
    lab.close()

TCLab version 0.4.9
Arduino Leonardo connected on port COM3 at 115200 baud.
TCLab Firmware 1.4.3 Arduino Leonardo/Micro.
T1 = 36.7, T2 = 32.19
TCLab disconnected successfully.
